In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torchvision import datasets, transforms

import numpy as np

In [52]:
train_dataset = FashionMNIST(root=".", train=True, download=True, transform=transforms.ToTensor())
test_dataset = FashionMNIST(root=".", train=False, download=True, transform=transforms.ToTensor())

In [56]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [177]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.class_size = 10
        self.conv_layer_1 =  nn.Conv2d(1, 14, kernel_size=11, stride=2, padding=1, dilation=1)
        #self.pool = nn.MaxPool2d(2, 2)
        # self.conv_layer_2 = nn.Conv2d(16, 10, (3, 3), 3, 3)
        self.conv_layer_2 = nn.Conv2d(14, 10, kernel_size=11, stride=2, padding=1, dilation=1)
        self.conv_layer_3 = nn.Conv2d(10, 10, kernel_size=1)
        
    def forward(self, input_image):
        result_1 = F.relu(self.conv_layer_1(input_image))
        #print(f'{result_1.shape}: {result_1}')
        result_2 = F.relu(self.conv_layer_2(result_1))
        result_3 = F.relu(self.conv_layer_3(result_2))
        #return F.softmax(result_3, dim=1).squeeze(2).squeeze(2)
        return F.log_softmax(result_3, dim=1).squeeze(2).squeeze(2)
    
class ModelMask(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.class_size = 10
        self.conv_layer_1 =  nn.Conv2d(1, 14, kernel_size=11, stride=2, padding=1, dilation=1)
        #self.pool = nn.MaxPool2d(2, 2)
        # self.conv_layer_2 = nn.Conv2d(16, 10, (3, 3), 3, 3)
        self.conv_layer_2 = nn.Conv2d(14, 10, kernel_size=11, stride=2, padding=1, dilation=1)
        self.conv_layer_3 = nn.Conv2d(10, 10, kernel_size=1)
        
    def forward(self, input_image):
        result_1 = F.relu(self.conv_layer_1(input_image))
        #print(f'{result_1.shape}: {result_1}')
        result_2 = F.relu(self.conv_layer_2(result_1))
        result_3 = F.relu(self.conv_layer_3(result_2))
        #return F.softmax(result_3, dim=1).squeeze(2).squeeze(2)
        return F.log_softmax(result_3, dim=1).squeeze(2).squeeze(2)
    

In [178]:
device = 0

model = Model().to(device)

net = Net().to(device)

In [179]:

# model.forward(image.view(28, 28, 1, 1))
#a = model.forward(image.view(-1))
a = model(train_dataset[0][0].view(1, 1, 28, 28).to(device))
#pixels = np.asarray(train_dataset[0][0])


In [180]:
a.shape

torch.Size([1, 10])

In [181]:
a.squeeze(2).squeeze(2).shape

RuntimeError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [185]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # print(f'CICCIO {output.shape}, {target.shape}')
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    use_cuda = 0
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=100, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
])),
        batch_size=10, shuffle=True, **kwargs)


    model = Model().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9, nesterov=True)

    for epoch in range(1, 10 + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304036
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.712879
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.583876
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.416198
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.305618
Train Epoch: 1 [25000/60000 (42%)]	Loss: 0.458283
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.550277
Train Epoch: 1 [35000/60000 (58%)]	Loss: 0.375713
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.436687
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.602333
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.282566
Train Epoch: 1 [55000/60000 (92%)]	Loss: 0.238015

Test set: Average loss: 0.3306, Accuracy: 8740/10000 (87%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.354596
Train Epoch: 2 [5000/60000 (8%)]	Loss: 0.337276
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.188028
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.423722
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.325596
Train Epoch: 2 [25000/60000 (42%)]	Loss: 0.307845
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.235111
Tra

In [ ]:
    l = datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))]))

In [63]:
l

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ../data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.1307,), std=(0.3081,))
                         )
    Target Transforms (if any): None

In [66]:
l[0][0].shape

torch.Size([1, 28, 28])

In [9]:
cap = datasets.CocoDetection(root = '/home/none/datasets/coco/train2017',
                        annFile = '/home/none/datasets/annotations/stuff_train2017.json',
                        transform=transforms.ToTensor())

loading annotations into memory...
Done (t=8.85s)
creating index...
index created!


In [10]:
s = None
for i in range(len(cap)):
    try:
        s = cap[i]
    except BaseException as e:
        print(f'CC is {i} {e}')
        ()
    if s is not None:
        break
        

CC is 0 [Errno 2] No such file or directory: '/home/none/datasets/coco/train2017/000000391895.jpg'
CC is 1 [Errno 2] No such file or directory: '/home/none/datasets/coco/train2017/000000522418.jpg'
CC is 2 [Errno 2] No such file or directory: '/home/none/datasets/coco/train2017/000000184613.jpg'
CC is 3 [Errno 2] No such file or directory: '/home/none/datasets/coco/train2017/000000318219.jpg'
CC is 4 [Errno 2] No such file or directory: '/home/none/datasets/coco/train2017/000000554625.jpg'
CC is 5 [Errno 2] No such file or directory: '/home/none/datasets/coco/train2017/000000574769.jpg'
CC is 6 [Errno 2] No such file or directory: '/home/none/datasets/coco/train2017/000000060623.jpg'
CC is 7 [Errno 2] No such file or directory: '/home/none/datasets/coco/train2017/000000309022.jpg'


In [18]:
s[1][0]['segmentation']['counts']

'c9n4Q:O10000000000O0101O0000O1001OO1010N100001N1O1WKnEd4S:[KoEd4X:M10O0100O1O010O100O00100O10O0100O1O010O100O10O10K^ElKc:g3^E`L1Ha:h3_E_L1H`:h3`E^L2J]:h3dE[Ld:d3]E[Le:d3\\EZLc:g3=M00100O001O10O0100O2N10kDbLi:_3UEcLS:M\\F_3@dLS:0\\F]3_OeLS:O_F^3ZOdLV:O`Fd3^9`LbF_3]9bLbF^3`9aLaF^3b9`L\\Fa3e9^L[Fb3e9^L[Fb3e9_LZF`3g9`LYF_3h9aLYF_3f9aLZF`3e9`LZFX3_OaLS:0RF8<W3e9aLoE8;W3g9aLnE99U3j9cLmE97Q3Q:eLhE97S3Z:PMbE[2HhMNAi:d0XES22hMMAi:^3ZEcL08MGa:0aE2N[3;cLE4e:a3jE[LX:e3`014K4M1O0OGdLQE\\3o:dLQE\\3o:dLQE[3P;eLPE[3Z;0000O100F_DVMa;j2`DTMa;k2bDSM^;g2`0bMUDh1k;XNWDb1m;]NUD_1n;aNRD\\1P<eNPDZ1Q<eNQDZ1o;fNQDZ1o;fNRDX1k;lNVDS1f;POcD_OXOU1S<^OkD`0T;AmD<U;DmD3mNYO[<d0iD2Y;MjD1V;NkD2V;MkD2V;MjD@gNa0`<NkD2U;NkD2U;NjD3V;MkD2U;NkD2U;NkD2V;J`CE\\17`;JRUh40^lWKc0J6kA0P=`1_O2M5L3N2M3M3N2M3L4GkLbDZ3_;3000000O2O0000000O2M101O2M2O1O0O13KO21N1^OiD^MW;g1gETNZ:j1kESNZ:IeD0Of1R1_Nf:a1[E\\Ng:c1[1EiBiN\\=W17N10000O1O1\\Od0O1N3L3M3K5N2000Vjg1'

In [20]:
from pycocotools.coco import COCO

In [26]:
coco = COCO('/home/none/datasets/annotations/stuff_train2017.json')

loading annotations into memory...
Done (t=7.90s)
creating index...
index created!


In [34]:
mask = coco.annToMask(s[1][0])

In [32]:
s[1][0]

{'segmentation': {'counts': 'c9n4Q:O10000000000O0101O0000O1001OO1010N100001N1O1WKnEd4S:[KoEd4X:M10O0100O1O010O100O00100O10O0100O1O010O100O10O10K^ElKc:g3^E`L1Ha:h3_E_L1H`:h3`E^L2J]:h3dE[Ld:d3]E[Le:d3\\EZLc:g3=M00100O001O10O0100O2N10kDbLi:_3UEcLS:M\\F_3@dLS:0\\F]3_OeLS:O_F^3ZOdLV:O`Fd3^9`LbF_3]9bLbF^3`9aLaF^3b9`L\\Fa3e9^L[Fb3e9^L[Fb3e9_LZF`3g9`LYF_3h9aLYF_3f9aLZF`3e9`LZFX3_OaLS:0RF8<W3e9aLoE8;W3g9aLnE99U3j9cLmE97Q3Q:eLhE97S3Z:PMbE[2HhMNAi:d0XES22hMMAi:^3ZEcL08MGa:0aE2N[3;cLE4e:a3jE[LX:e3`014K4M1O0OGdLQE\\3o:dLQE\\3o:dLQE[3P;eLPE[3Z;0000O100F_DVMa;j2`DTMa;k2bDSM^;g2`0bMUDh1k;XNWDb1m;]NUD_1n;aNRD\\1P<eNPDZ1Q<eNQDZ1o;fNQDZ1o;fNRDX1k;lNVDS1f;POcD_OXOU1S<^OkD`0T;AmD<U;DmD3mNYO[<d0iD2Y;MjD1V;NkD2V;MkD2V;MjD@gNa0`<NkD2U;NkD2U;NjD3V;MkD2U;NkD2U;NkD2V;J`CE\\17`;JRUh40^lWKc0J6kA0P=`1_O2M5L3N2M3M3N2M3L4GkLbDZ3_;3000000O2O0000000O2M101O2M2O1O0O13KO21N1^OiD^MW;g1gETNZ:j1kESNZ:IeD0Of1R1_Nf:a1[E\\Ng:c1[1EiBiN\\=W17N10000O1O1\\Od0O1N3L3M3K5N2000Vjg1',
  'size': [479, 640]},
 'area': 20396.0,
 'iscrowd':

In [ ]:
mask.sum()